# Lab session 6: Association Analysis

## Introduction

The purpose of this lab session is to provide you with an opportunity to gain experience in **association analysis** using typical Python libraries.

- This lab is the first part of a **two-week assignment** that covers weeks 7 and 8, which is due on **Tuesday 24th November 10am**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet will contribute to 5% of your overall grade; questions in the lab sheet for week 8 will cover for another 5% of your overall grade.
- <font color = 'maroon'>The last section of this notebook includes the questions that are assessed towards your final grade.</font> 

This session starts with a tutorial that uses examples to introduce you to the practical knowledge that you will need for the corresponding assignment. We highly recommend that you read the following tutorials if you need a gentler introduction to the libraries that we use:
- [Mlxtend: Apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
- [Numpy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html)
- [Numpy: basic broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- [Matplotlib](https://matplotlib.org/tutorials/introductory/pyplot.html)
- [Seaborn](https://seaborn.pydata.org/tutorial/relational.html)
- [Scikit-learn](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

## Important notes about the assignment: 

- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- You are asked to submit a **report** that should answer the questions specified in the last section of this notebook. The report should be in **PDF format** (so **NOT** *doc, docx, notebook* etc). It should be well identified with your name, student number, assignment number (for instance, Assignment 3), module, and marked with question numbers. 
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment3-StudentName-StudentNumber.pdf
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.

## 1. Frequent itemsets

In order to present functionalities for association analysis in Python, we adapt an example from the ``mlxtend`` documentation.

Consider a dataset composed of five transactions. This dataset is represented by a list of five elements, each of which is a list of items bought during a trip to a supermarket.

In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

The library ``mlxtend`` requires that each transaction is represented by a binary vector where each element indicates the presence or absence of a specific item.

The method ``TransactionEncoder.fit_transform`` can be used to convert the dataset created above into this expected format. This method returns a binary matrix (numpy array) where each transaction corresponds to a row and each column corresponds to an item.

In [2]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
print(te_ary)

[[False False False  True False  True  True  True  True False  True]
 [False False  True  True False  True False  True  True False  True]
 [ True False False  True False  True  True False False False False]
 [False  True False False False  True  True False False  True  True]
 [False  True False  True  True  True False False  True False False]]


The item corresponding to each column is stored by the ``TransactionEncoder`` object in a variable called ``columns_``. This variable can be used to create a ``DataFrame`` that conveniently represents the transaction dataset.

In [3]:
import pandas as pd

df = pd.DataFrame(te_ary, columns=te.columns_)
display(df)

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


The ``mlxtend`` function ``apriori`` receives a ``DataFrame`` that represents a transaction dataset and a parameter that specifies the support threshold. This function returns a ``DataFrame`` that contains one row for each frequent itemset. Each row contains a python ``frozenset`` that represents the itemset (by column indices) and a number that represents the support of this itemset.

In [4]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.6)
display(frequent_itemsets)

itemset = frequent_itemsets.loc[5]
print('Itemset: {0}. Support: {1}.'.format(itemset['itemsets'], itemset['support']))

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


Itemset: frozenset({3, 5}). Support: 0.8.


Conveniently, if the parameter ``use_colnames`` is set to ``True``,  the ``mlxtend`` function ``apriori`` may instead return a ``DataFrame`` that represents itemsets by ``frozensets`` of item names.

In [5]:
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
display(frequent_itemsets)

,support,itemsets
0,0.2,(Apple)
1,0.4,(Corn)
2,0.2,(Dill)
3,0.8,(Eggs)
4,0.2,(Ice cream)
...,...,...
144,0.4,"(Onion, Nutmeg, Kidney Beans, Eggs, Yogurt)"
145,0.2,"(Onion, Nutmeg, Milk, Eggs, Yogurt)"
146,0.2,"(Onion, Nutmeg, Kidney Beans, Milk, Yogurt)"
147,0.2,"(Onion, Nutmeg, Kidney Beans, Dill, Eggs, Yogurt)"


Using typical ``pandas`` functionalities, it is easy to include a column in such a ``DataFrame`` to register the number of items in each frequent itemset, which can be used to filter itemsets by length.

In [6]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # length of each frozenset
print('Frequent 3-itemsets:')
display(frequent_itemsets[frequent_itemsets['length'] == 3])

Frequent 3-itemsets:


,support,itemsets,length
47,0.2,"(Eggs, Kidney Beans, Apple)",3
48,0.2,"(Eggs, Milk, Apple)",3
49,0.2,"(Kidney Beans, Apple, Milk)",3
50,0.2,"(Eggs, Corn, Ice cream)",3
51,0.2,"(Eggs, Corn, Kidney Beans)",3
52,0.2,"(Onion, Eggs, Corn)",3
53,0.2,"(Corn, Kidney Beans, Ice cream)",3
54,0.2,"(Onion, Corn, Ice cream)",3
55,0.2,"(Corn, Kidney Beans, Milk)",3
56,0.2,"(Onion, Corn, Kidney Beans)",3


In [7]:
display(frequent_itemsets)

,support,itemsets,length
0,0.2,(Apple),1
1,0.4,(Corn),1
2,0.2,(Dill),1
3,0.8,(Eggs),1
4,0.2,(Ice cream),1
...,...,...,...
144,0.4,"(Onion, Nutmeg, Kidney Beans, Eggs, Yogurt)",5
145,0.2,"(Onion, Nutmeg, Milk, Eggs, Yogurt)",5
146,0.2,"(Onion, Nutmeg, Kidney Beans, Milk, Yogurt)",5
147,0.2,"(Onion, Nutmeg, Kidney Beans, Dill, Eggs, Yogurt)",6


It is also easy to create a ``dict`` that maps any frequent itemset (represented by a ``frozenset``) to its support.

In [8]:
support = {}
for _, row in frequent_itemsets.iterrows():
    support[row['itemsets']] = row['support']

itemset = frozenset(['Onion', 'Eggs'])
print('Itemset: {0}. Support: {1}.'.format(itemset, support[itemset]))

Itemset: frozenset({'Onion', 'Eggs'}). Support: 0.6.


## 2. Association rules

The ``mlxtend`` function ``association_rules`` receives a ``DataFrame`` that represents the set of frequent itemsets and returns a ``DataFrame`` that represents strong association rules for a specified confidence threshold. Each row in the resulting  ``DataFrame`` contains an association rule together with some potentially useful measures (we have not covered lift, leverage, or conviction). 

In [9]:
from mlxtend.frequent_patterns import association_rules

strong_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
display(strong_rules[15:17])
#print(strong_rules['antecedents'])
#print(type(strong_rules['antecedents']))

#print((strong_rules.iloc[0]['antecedents']))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
16,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6


## <font color = 'maroon'>Assignment 3 [Part 1 of 2]</font>

Questions 7-8 are coding exercises. Questions 1-6 do not require you to write any code. In all responses, please show your workings (equations, justifications, or code when applicable).

1. What is the advantage of using the Apriori algorithm in comparison with computing the support of every subset of an itemset in order to find the frequent itemsets in a transaction dataset? [0.5 marks out of 5]
2. Let $\mathcal{L}_1$ denote the set of frequent $1$-itemsets. For $k \geq 2$, why must every frequent $k$-itemset be a superset of an itemset in $\mathcal{L}_1$? [0.5 marks out of 5]
3. Let $\mathcal{L}_2 = \{ \{1,2\}, \{1,5\}, \{2, 3\}, \{3, 4\}, \{3, 5\}\}$. Compute the set of candidates $\mathcal{C}_3$ that is obtained by joining every pair of joinable itemsets from $\mathcal{L}_2$. [0.5 marks out of 5]
4. Let $S_1$ denote the support of the association rule $\{ \text{popcorn, soda} \} \Rightarrow \{ \text{movie} \}$. Let $S_2$ denote the support of the association rule $\{ \text{popcorn} \} \Rightarrow \{ \text{movie} \}$. What is the relationship between $S_1$ and $S_2$? [0.5 marks out of 5]
5. What is the support of the rule $\{  \} \Rightarrow \{ \text{Kidney Beans} \}$ in the transaction dataset used in the tutorial presented above? [0.5 marks out of 5]
6. In the transaction dataset used in the tutorial presented above, what is the maximum length of a frequent itemset for a support threshold of 0.2? [0.5 marks out of 5]
7. Implement a function that receives a ``DataFrame`` of frequent itemsets and a **strong** association rule (represented by a ``frozenset`` of antecedents and a ``frozenset`` of consequents). This function should return the corresponding Kulczynski measure. Include the code in your report. [1 mark out of 5]
8. Implement a function that receives a ``DataFrame`` of frequent itemsets and a **strong** association rule (represented by a ``frozenset`` of antecedents and a ``frozenset`` of consequents). This function should return the corresponding imbalance ratio. Include the code in your report. [1 mark out of 5]


# Answers to Coding Questions:

7. Implement a function that receives a ``DataFrame`` of frequent itemsets and a **strong** association rule (represented by a ``frozenset`` of antecedents and a ``frozenset`` of consequents). This function should return the corresponding Kulczynski measure. Include the code in your report. [1 mark out of 5]


In [10]:
def calculate_kulczynski(itemset, antecendents, consequents):
    # We don't need to set a threshold as the question states that we would pick 
    # strong rules by ourselves. (Otherwise we could set threshold; give one set of
    #antecendents and consequents, ) 
    rules = association_rules(itemset, metric="confidence", min_threshold=0) 
    
    # Forward & Backward Rule: - Assumed always to return a strong rule
    fwd_rule = rules[(rules['antecedents'] == antecendents) & (rules['consequents'] == consequents)]
    bck_rule = rules[(rules['antecedents'] == consequents) & (rules['consequents'] == antecendents)]
    
    # Confidences:
    conf_fwd = fwd_rule['confidence'].values[0]
    #print(conf_fwd)
    conf_bck = bck_rule['confidence'].values[0]
    #print(conf_bck)
    
    kul = (conf_bck + conf_fwd) / 2
    #print(kul)
    
    return kul

In [11]:
# Do two tests: 

# Test 1: (eggs -> onion) - Both ways are strong! 0.7
ants = frozenset(['Eggs'])
cons = frozenset(['Onion'])
print('Test 1: ' + str(calculate_kulczynski(frequent_itemsets, ants, cons)))

# Test 2: (eggs, corn -> ice cream) - Both ways are strong! 0.7
ants = frozenset(['Eggs', 'Corn'])
cons = frozenset(['Ice cream'])
print('Test 2: ' + str(calculate_kulczynski(frequent_itemsets, ants, cons)))

Test 1: 0.875
Test 2: 1.0


8. Implement a function that receives a ``DataFrame`` of frequent itemsets and a **strong** association rule (represented by a ``frozenset`` of antecedents and a ``frozenset`` of consequents). This function should return the corresponding imbalance ratio. Include the code in your report. [1 mark out of 5]



In [12]:
def calculate_imbalance_ratio(itemset, antecendents, consequents):
    """
    Takes a DataFrame of frequent itemsets and a strong association rule (Antacedents & Consequents).
    Returns imbalance ratio. 
    """

    ab_set = frozenset.union(antecendents,consequents)
    
    a = itemset[itemset['itemsets'] == antecendents]
    b = itemset[itemset['itemsets'] == consequents]
    ab = itemset[itemset['itemsets'] == ab_set]
    
    # Actually calculating the N factor is not necessary 
    # as it will be gone when calculating the ratio.
    support_a = a['support'].values[0] 
    print(support_a)
    support_b = b['support'].values[0] 
    print(support_b)
    support_ab = ab['support'].values[0] 
    print(support_ab)
    
    return abs(support_a-support_b) / (support_a+support_b-support_ab)

In [13]:
# Do two tests: - Same as previous section:

# Test 1: (eggs -> onion) - Both ways are strong! 0.7
ants = frozenset(['Eggs'])
cons = frozenset(['Onion'])
print('Test 1: ' + str(calculate_imbalance_ratio(frequent_itemsets, ants, cons)))

# Test 2: (eggs, corn -> ice cream) - Both ways are strong! 0.7
ants = frozenset(['Eggs', 'Corn'])
cons = frozenset(['Ice cream'])
print('Test 2: ' + str(calculate_imbalance_ratio(frequent_itemsets, ants, cons)))

0.8
0.6
0.6
Test 1: 0.2500000000000001
0.2
0.2
0.2
Test 2: 0.0
